In [57]:
%matplotlib inline
import numpy as np
from scipy import signal
from scipy import ndimage #can't belive numpy and base scipy don't have circular convolution
from matplotlib import pyplot as plt

In [166]:
N_points = 500
x = 2*np.pi*(np.linspace(0,1-1./N_points,N_points)+.5/N_points)
dx = np.ones_like(x)/len(x)

Algorithm for generating

In [167]:
def integr(f, mu = None): #default is lebesgue measure
    if mu is None:
        mu = dx
    return np.sum(f*mu)

def convolve(f,mu): #we will only convolve functions against measures, so no normalisation
    return ndimage.filters.convolve1d(f,mu,mode='wrap')

def diff(f, dmu=None): #derivative of the Radon-Nykodim derivative as a measure
                      #(default wrt dx)
    if dmu is None:
        dmu = dx
        
    df = np.zeros_like(f)
    df[:-1]+=f[1: ]; df[-1]+=f[0]
    df[1:] -=f[:-1]; df[0] -=f[-1]

    return df/dmu/2.

In [168]:
def dmudt(mu):
    #everything is more or less well defined in the sense of:
    # conv: (measure x function)-> function
    # times: (measure x function)-> measure
    # RN derivative: (measure x measure) -> measure (I'm allowing more than usual here!)
    f = 1/np.tan(x/2.)
    return -diff(convolve(f,mu)*mu)+mu
def euler_scheme(x0, f, step_size, n_steps):
    x = np.copy(x0)
    for i in range(n_steps):
        x+=step_size*f(x)
    return x

# 1-dimensional random measure

We define an analogous of the 2-dimensional regularization procedure for LQG in order to obtain a random noise measure on $S^1$ which has a similar conformal covariance structure to LQG. We want to define it in such a way that it is a locally independent random measure. Start as follows:

Let $\phi_\epsilon(t) = \partial_t ( \epsilon^{-1/2}\chi_{[0,\epsilon]} \ast W(t))$ that is, a regularized version of the derivative of the brownian motion, with covariance function $<\chi(x),\chi(y)>$. Take the exponential of that.